In [1]:
from pyasn1.type import univ, namedtype, namedval, tag, char
from pyasn1.codec.ber import encoder
from pyasn1.codec.ber import decoder
from trustmessages import *

In [2]:
sq1 = Query()
sq1["con"] = Constraint()
sq1["con"]["operator"] = "lt"
sq1["con"]["value"] = Value()
sq1["con"]["value"]["date"] = 50

In [3]:
sq2 = Query()
sq2["exp"] = Expression()
sq2["exp"]["operator"] = "and"
sq2["exp"]["left"] = Query()
sq2["exp"]["left"]["con"] = Constraint()
sq2["exp"]["left"]["con"]["operator"] = "eq"
sq2["exp"]["left"]["con"]["value"] = Value()
sq2["exp"]["left"]["con"]["value"]["source"] = "alice"
sq2["exp"]["right"] = Query()
sq2["exp"]["right"]["con"] = Constraint()
sq2["exp"]["right"]["con"]["operator"] = "eq"
sq2["exp"]["right"]["con"]["value"] = Value()
sq2["exp"]["right"]["con"]["value"]["target"] = "bob"

In [4]:
q = Query()
q["exp"] = Expression()
q["exp"]["left"] = Query()
q["exp"]["left"]["con"] = Constraint()
q["exp"]["left"]["con"]["operator"] = "eq"
q["exp"]["left"]["con"]["value"] = Value()
q["exp"]["left"]["con"]["value"]["service"] = "seller"

q["exp"]["operator"] = "and"

q["exp"]["right"] = Query()
q["exp"]["right"]["exp"] = Expression()
q["exp"]["right"]["exp"]["operator"] = "or"
q["exp"]["right"]["exp"]["left"] = Query()
q["exp"]["right"]["exp"]["left"]["con"] = Constraint()
q["exp"]["right"]["exp"]["left"]["con"]["operator"] = "eq"
q["exp"]["right"]["exp"]["left"]["con"]["value"] = Value()
q["exp"]["right"]["exp"]["left"]["con"]["value"]["source"] = "charlie"
q["exp"]["right"]["exp"]["right"] = Query()
q["exp"]["right"]["exp"]["right"]["con"] = Constraint()
q["exp"]["right"]["exp"]["right"]["con"]["operator"] = "eq"
q["exp"]["right"]["exp"]["right"]["con"]["value"] = Value()
q["exp"]["right"]["exp"]["right"]["con"]["value"]["source"] = "david"

In [5]:
substrate = encoder.encode(q)
d, e = decoder.decode(substrate, asn1Spec=Query())
assert(d == q)
assert(q.prettyPrint() == d.prettyPrint())

In [6]:
qtm = QtmDb()

In [7]:
assert(all(t["date"] < 50
           for t in filter(trustutils.create_predicate(sq1), qtm.trust_db)))

In [8]:
assert(all(t["source"] == "alice" and t["target"] == "bob"
           for t in filter(trustutils.create_predicate(sq2), qtm.assessment_db)))

In [9]:
assert(all(t["service"] == "seller" and (t["source"] == "charlie" or t["source"] == "david")
           for t in filter(trustutils.create_predicate(q), qtm.assessment_db)))